In [5]:
import numpy as np

# Константы из исходного кода
n = 10
m = 6

# Глобальные переменные (аналог var в Pascal)
x = np.zeros(n+1)
y = np.zeros(n+1)
id = np.zeros(n+1)
xi = np.zeros(n+1)
zi = np.zeros(n+1)
a = np.zeros(m+1)  # индексы в Python начинаются с 0, поэтому +1 для соответствия
hh = np.zeros(5)   # массивы 0..4 в Pascal → 0..4 в Python
xx = np.zeros(5)

# Функция реализует метод Гаусса с выбором главного элемента
def gauss(nl, a_matrix, x_vector):
    pl = list(range(nl+1))  # индексация с 1 в Pascal → 0..nl в Python
    sol = True
    
    # Прямой ход
    for step in range(1, nl):
        abed = 0.0
        nbed = step
        
        # Выбор главного элемента
        for il in range(step, nl+1):
            if abs(a_matrix[pl[il]][step]) > abed:
                nbed = il
                abed = abs(a_matrix[pl[il]][step])
        
        if abed == 0:
            sol = False
            break
        
        # Перестановка строк
        pl[step], pl[nbed] = pl[nbed], pl[step]
        
        # Исключение переменных
        for lin in range(step+1, nl+1):
            r = a_matrix[pl[lin]][step] / a_matrix[pl[step]][step]
            for st in range(step+1, nl+2):
                a_matrix[pl[lin]][st] -= a_matrix[pl[step]][st] * r
            a_matrix[pl[lin]][step] = 0.0
    
    # Обратный ход
    if sol:
        x_vector[nl] = a_matrix[pl[nl]][nl+1] / a_matrix[pl[nl]][nl]
        for lin in range(nl-1, 0, -1):
            f = sum(-a_matrix[pl[lin]][st] * x_vector[st] for st in range(lin+1, nl+1))
            x_vector[lin] = (a_matrix[pl[lin]][nl+1] + f) / a_matrix[pl[lin]][lin]
    
    return sol

# Метод наименьших квадратов
def mnk(m_val, n_val, d, fi, x_vec):
    aa = np.zeros((m_val+1, m_val+2))  # m x (m+1) матрица
    
    # Заполнение матрицы системы
    for i in range(1, m_val+1):
        aa[i][m_val+1] = sum(d[l] * fi[i][l] for l in range(1, n_val+1))
        for j in range(1, m_val+1):
            aa[i][j] = sum(fi[i][l] * fi[j][l] for l in range(1, n_val+1))
    
    # Решение СЛАУ методом Гаусса
    gauss(m_val, aa, x_vec)

# Кубические базисные функции Эрмита
def sp(t, i):
    tt = t**2
    if i == 1:
        return 1 - 3*tt + 2*tt*t
    elif i == 2:
        return 3*tt - 2*tt*t
    elif i == 3:
        return t*(1 - t)**2
    elif i == 4:
        return -t**2*(1 - t)

# Производные базисных функций
def spx(t, i):
    tt = t**2
    if i == 1:
        return -6*t + 6*tt
    elif i == 2:
        return 6*t - 6*tt
    elif i == 3:
        return 1 - 4*t + 3*tt
    elif i == 4:
        return -2*t + 3*tt

# Вторые производные
def spxx(t, i):
    if i == 1:
        return -6 + 12*t
    elif i == 2:
        return 6 - 12*t
    elif i == 3:
        return -4 + 6*t
    elif i == 4:
        return -2 + 6*t

# Функции для B-сплайнов 3-го порядка
def phi(t, i):
    abs_t = abs(t)
    if abs_t > 1:
        return 0.0
    if i == 1:
        return (abs_t - 1)**2 * (2*abs_t + 1)
    elif i == 2:
        return t * (abs_t - 1)**2

# Вторые производные B-сплайнов
def phixx(t, i):
    abs_t = abs(t)
    if abs_t > 1:
        return 0.0
    if i == 1:
        return 12*abs_t - 6
    elif i == 2:
        return (-4 + 6*t) if t > 0 else (4 + 6*t)

# Определение функции pfr4
def pfr4(x):
    x2 = x * x
    x3 = x2 * x
    if x < 0:
        return 0.0
    elif 0 <= x < 1:
        return x3 / 6
    elif 1 <= x < 2:
        return (-x3 / 2) + 2 * x2 - 2 * x + 2/3
    elif 2 <= x < 3:
        return (x3 / 2) - 4 * x2 + 10 * x - 22/3
    elif 3 <= x < 4:
        return (-x3 / 6) + 2 * x2 - 8 * x + 32/3
    else:
        return 0.0

# Основная часть программы
if __name__ == "__main__":
    # Инициализация данных
    xx = np.array([0.0, 1.0, 2.0, 3.0, 4.0])
    hh = np.diff(xx)  # шаги сетки
    
    # Чтение данных из файла (заменено на тестовые данные)
    x = np.linspace(-0.2, 0.2*10, n)
    y = np.array([pfr4(xi) for xi in x])  # pfr4 - функция из исходного кода
    
    # Формирование базисных функций
    fi = np.zeros((m+1, n+1))  # m=6, n=10
    for i in range(1, n+1):
        t = x[i]
        # Вызов процедуры zh3 (аналог в Python)
        for i1, j in [(1,1), (2,3), (3,5)]:
            if t < xx[i1]:
                fi[j][i] = phi((t - xx[i1])/hh[i1-1], 1)
                fi[j+1][i] = hh[i1-1] * phi((t - xx[i1])/hh[i1-1], 2)
            else:
                fi[j][i] = phi((t - xx[i1])/hh[i1], 1)
                fi[j+1][i] = hh[i1] * phi((t - xx[i1])/hh[i1], 2)
    
    # Решение МНК
    mnk(m, n, y, fi, a)
    
    # Вывод результатов
    for i in range(1, m+1):
        print(f"{i:5d} {a[i]:9.5f}")
    
    # Табуляция функции
    for i in range(41):
        t = i / 10.0
        s = sum(a[j] * phi(t - xx[(j-1)//2 + 1], 1 if j%2 else 2) for j in range(1, m+1))
        print(f"{t:8.2f} {s:8.4f} {pfr4(t):8.4f}")

IndexError: index 10 is out of bounds for axis 0 with size 10